https://python.langchain.com/docs/use_cases/sql/quickstart/

### **Test the sqldb**

In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

**Connecting to the sqldb**

In [2]:
db_path = str(here("data")) + "/sqldb.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

In [3]:
db

In [4]:
# validate the connection to the vectordb
print(db.dialect)

sqlite


In [5]:
print(db.get_usable_table_names())

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [6]:

db._execute("SELECT * FROM Artist LIMIT 10;")

[{'ArtistId': 1, 'Name': 'AC/DC'},
 {'ArtistId': 2, 'Name': 'Accept'},
 {'ArtistId': 3, 'Name': 'Aerosmith'},
 {'ArtistId': 4, 'Name': 'Alanis Morissette'},
 {'ArtistId': 5, 'Name': 'Alice In Chains'},
 {'ArtistId': 6, 'Name': 'Antônio Carlos Jobim'},
 {'ArtistId': 7, 'Name': 'Apocalyptica'},
 {'ArtistId': 8, 'Name': 'Audioslave'},
 {'ArtistId': 9, 'Name': 'BackBeat'},
 {'ArtistId': 10, 'Name': 'Billy Cobham'}]

### **Test the access to the environment variables**

In [ ]:
import os
from langchain_openai import ChatOpenAI


os.environ['GITHUB_TOKEN'] = ""  # Replace with your actual GitHub token
token = os.environ.get("GITHUB_TOKEN")
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4.1-mini" 

if not token:
    raise ValueError("GITHUB_TOKEN environment variable not set. Please provide a valid token.")

llm =ChatOpenAI(
    model_name=model_name,
    openai_api_key=token,
    openai_api_base=endpoint,
    temperature=0.5,
)

### **Test your GPT model**

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage
response = llm.invoke([
    HumanMessage(content="What's the capital of Bangladesh?")
])

print(response.content)

The capital of Bangladesh is Dhaka.


In [10]:
response

AIMessage(content='The capital of Bangladesh is Dhaka.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 13, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CbdJmzT5Lqsv54uNsMoGGn1ZicSNn', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d7355bf0-e832-4d7b-a7c3-d8834cc81852-0', usage_metadata={'input_tokens': 13, 'output_tokens': 9, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### **1. SQL query chain**

In [17]:
# In newer langchain versions (1.0+), create_sql_query_chain is not available
# We'll create a simple chain using prompts and the LLM directly

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create a prompt template for SQL query generation
sql_prompt = ChatPromptTemplate.from_template("""
Given the database schema below, write a SQL query that answers the user's question.

Database Schema:
{schema}

Question: {question}

SQL Query (return ONLY the SQL query, no explanation):
""")

# Get database schema
schema_info = db.get_table_info()

# Create the chain
chain = (
    RunnablePassthrough.assign(schema=lambda _: schema_info)
    | sql_prompt
    | llm
    | StrOutputParser()
)

response = chain.invoke({"question": "How many employees are there"})
print(response)

```sql
SELECT COUNT(*) AS EmployeeCount FROM Employee;
```


In [20]:
import re
def extract_sql_query_simple(response):
    """
    Extract SQL query from various response formats:
    1. Markdown code blocks: ```sql ... ```
    2. SQLQuery: label format
    3. Plain SQL query
    """
    # First try to extract from markdown code block (```sql ... ```)
    code_block_match = re.search(r'```sql\s*(.*?)\s*```', response, re.IGNORECASE | re.DOTALL)
    if code_block_match:
        sql_query = code_block_match.group(1).strip()
        return sql_query
    
    # Try to find SQL query after "SQLQuery:" label
    label_match = re.search(r"SQLQuery:\s*(.*?)(?:\n|$)", response, re.IGNORECASE)
    if label_match:
        sql_query = label_match.group(1).strip()
        return sql_query
    
    # If response looks like a plain SQL query (starts with SELECT, INSERT, UPDATE, DELETE, etc.)
    plain_sql_match = re.match(r'^\s*(SELECT|INSERT|UPDATE|DELETE|CREATE|ALTER|DROP)\s+', 
                                response, re.IGNORECASE | re.DOTALL)
    if plain_sql_match:
        return response.strip()
    
    return None

In [21]:
# Test the extraction function with different formats
test_responses = [
    # Format 1: Markdown code block
    """```sql
SELECT COUNT(*) AS EmployeeCount FROM Employee;
```""",
    # Format 2: SQLQuery label
    "SQLQuery: SELECT * FROM Artist LIMIT 10;",
    # Format 3: Plain SQL
    "SELECT COUNT(*) FROM Employee;"
]

print("Testing extract_sql_query_simple function:")
for i, test in enumerate(test_responses, 1):
    result = extract_sql_query_simple(test)
    print(f"\nTest {i}:")
    print(f"Input: {test[:50]}...")
    print(f"Extracted: {result}")

Testing extract_sql_query_simple function:

Test 1:
Input: ```sql
SELECT COUNT(*) AS EmployeeCount FROM Emplo...
Extracted: SELECT COUNT(*) AS EmployeeCount FROM Employee;

Test 2:
Input: SQLQuery: SELECT * FROM Artist LIMIT 10;...
Extracted: SELECT * FROM Artist LIMIT 10;

Test 3:
Input: SELECT COUNT(*) FROM Employee;...
Extracted: SELECT COUNT(*) FROM Employee;


In [22]:
query=extract_sql_query_simple(response)
print(query)

SELECT COUNT(*) AS EmployeeCount FROM Employee;


Execute the query to make sure it’s valid

In [23]:
db.run(query)

'[(8,)]'

In [24]:
db._execute(query)

[{'EmployeeCount': 8}]

In [25]:
chain.get_prompts()[0].pretty_print()

================================ Human Message =================================


Given the database schema below, write a SQL query that answers the user's question.

Database Schema:
{schema}

Question: {question}

SQL Query (return ONLY the SQL query, no explanation):



### **Add QuerySQLDataBaseTool to the chain**
Execute SQL query

**This is the most dangerous part of creating a SQL chain.** Consider carefully if it is OK to run automated queries over your data. Minimize the database connection permissions as much as possible. Consider adding a human approval step to you chains before query execution (see below).

We can use the QuerySQLDatabaseTool to easily add query execution to our chain:

In [ ]:
# userquestion->create_sql_query_chain->response->Regex(extract_sql_query_simple)->query->db.run(query)->db._execute(query)

In [27]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# Use our custom SQL query generation chain (defined earlier)
# Recreate the write_query chain for clarity
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_prompt = ChatPromptTemplate.from_template("""
Given the database schema below, write a SQL query that answers the user's question.

Database Schema:
{schema}

Question: {question}

Return the SQL query in a markdown code block like this:
```sql
YOUR_QUERY_HERE
```
""")

schema_info = db.get_table_info()

write_query = (
    RunnablePassthrough.assign(schema=lambda _: schema_info)
    | sql_prompt
    | llm
    | StrOutputParser()
)

execute_query = QuerySQLDataBaseTool(db=db)

# Chain: question -> generate SQL -> extract SQL -> execute SQL
chain = write_query | extract_sql_query_simple | execute_query

chain.invoke({"question": "How many employees are there"})

'[(8,)]'

In [28]:
chain.invoke({"question": "Give me the names of all employees in the company (dont use any limit)"})

"[('Andrew', 'Adams'), ('Nancy', 'Edwards'), ('Jane', 'Peacock'), ('Margaret', 'Park'), ('Steve', 'Johnson'), ('Michael', 'Mitchell'), ('Robert', 'King'), ('Laura', 'Callahan')]"

In [29]:
chain.invoke({"question": "GIve me all employee title ? (dont use any limit)"})

"[('General Manager',), ('Sales Manager',), ('Sales Support Agent',), ('IT Manager',), ('IT Staff',)]"

### **Answer the question in a user friendly manner**

In [30]:
# Create a chain that generates SQL queries and extracts them
query_generation_chain = write_query | extract_sql_query_simple

In [31]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}

Answer: """
)

answer = answer_prompt | llm | StrOutputParser()

query = query_generation_chain.invoke({"question": "How many employees are there"})
sql_query_response = db._execute(query)

answer.invoke({"question": "How many employees are there","query": query, "result": sql_query_response})

'There are 8 employees.'

In [32]:
query = query_generation_chain.invoke({"question": "Give me the names of all employees in the company (dont use any limit)"})
sql_query_response = db._execute(query)

answer.invoke({"question": "Give me the names of all employees in the company (dont use any limit)","query": query, "result": sql_query_response})

'The names of all employees in the company are:\n\n- Andrew Adams  \n- Nancy Edwards  \n- Jane Peacock  \n- Margaret Park  \n- Steve Johnson  \n- Michael Mitchell  \n- Robert King  \n- Laura Callahan'

### **2. Agents**

Agent which provides a more flexible way of interacting with SQL databases. The main advantages of using the SQL Agent are:

- It can answer questions based on the databases’ schema as well as on the databases’ content (like describing a specific table).
- It can recover from errors by running a generated query, catching the traceback and regenerating it correctly.
- It can answer questions that require multiple dependent queries.
- It will save tokens by only considering the schema from relevant tables.

To initialize the agent, we use create_sql_agent function. This agent contains the SQLDatabaseToolkit which contains tools to:

- Create and execute queries
- Check query syntax
- Retrieve table descriptions
- …

In [33]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [34]:
agent_executor.invoke(
    {
        "input": "List the total sales per country. Which country's customers spent the most?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Count

{'input': "List the total sales per country. Which country's customers spent the most?",
 'output': 'The total sales per country for the top 10 countries are as follows:\n- USA: 523.06\n- Canada: 303.96\n- France: 195.1\n- Brazil: 190.1\n- Germany: 156.48\n- United Kingdom: 112.86\n- Czech Republic: 90.24\n- Portugal: 77.24\n- India: 75.26\n- Chile: 46.62\n\nThe customers from the USA spent the most.'}

In [35]:
agent_executor.invoke({"input": "Describe the playlisttrack table"})
# agent_executor.invoke("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Invoking: `sql_db_schema` with `{'table_names': 'PlaylistTrack'}`



CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("

{'input': 'Describe the playlisttrack table',
 'output': 'The PlaylistTrack table has two columns: PlaylistId and TrackId. Both columns are integers and together they form the primary key of the table. The PlaylistId column references the Playlist table, and the TrackId column references the Track table. The table represents the relationship between playlists and tracks, indicating which tracks belong to which playlists.'}

In [36]:
agent_executor.invoke({"input": "Tell me about the Artist table. What is the primary key?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Artist'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_schema` with `{'table_names': 'Artist'}`



CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/The Artist table has two columns: ArtistId and Name. The primary key of the Artist table is ArtistId.

> Finished chain.
The Artist table has two columns: ArtistId and

{'input': 'Tell me about the Artist table. What is the primary key?',
 'output': 'The Artist table has two columns: ArtistId and Name. The primary key of the Artist table is ArtistId.'}